In [1]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
DRIVE_MOUNT = True
 OTPUT_CLIP_PATH = "/content/drive/MyDrive/new_output.mp4"
# INPUT_PERSON_IMAGE =

CROPS_DIR = "/content/drive/MyDrive/tracks_crops"

MOT_RESULTS_PATH = "/content/drive/MyDrive/mot_results.txt" # tracker output for MOT evaluation
GROUND_TRUTH_MOT = "/content/drive/MyDrive/gt.txt" # optional: ground truth in MOT format for evaluation


YOLO_WEIGHTS = "yolov8n.pt" # change to your custom weights path if needed
DETECTION_CONFIDENCE = 0.3
MAX_COSINE_DISTANCE = 0.3
OUTPUT_FPS = 25


In [3]:
import torch

if torch.cuda.is_available():
    print("CUDA is available! Using GPU:", torch.cuda.get_device_name(0))
else:
    print("CUDA not available. Using CPU.")


CUDA is available! Using GPU: Tesla T4


In [4]:
!git clone https://github.com/KaiyangZhou/deep-person-reid.git
!cd deep-person-reid
# !conda create --name torchreid python=3.7
# !conda activate torchreid
!pwd

fatal: destination path 'deep-person-reid' already exists and is not an empty directory.
/content


In [5]:
import os
os.chdir("deep-person-reid")
print(os.getcwd())

/content/deep-person-reid


In [6]:
!pip install -r requirements.txt
!python setup.py develop

2025-12-09 09:14:33.195149: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765271673.215688    2307 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765271673.221621    2307 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1765271673.236354    2307 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765271673.236386    2307 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765271673.236389    2307 computation_placer.cc:177] computation placer alr

In [7]:
import torchreid
from torchreid.utils import FeatureExtractor
import torch
import numpy as np


In [8]:
# osnet_x1_0
extractor_x1 = FeatureExtractor(
    model_name='osnet_x1_0',
    model_path='',
    device='cuda' if torch.cuda.is_available() else 'cpu'
)

/content/deep-person-reid/torchreid/models/osnet.py:482: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(cached_file)


Successfully loaded imagenet pretrained weights from "/root/.cache/torch/checkpoints/osnet_x1_0_imagenet.pth"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']
Model: osnet_x1_0
- params: 2,193,616
- flops: 978,878,352


In [9]:
import os
import sys

try:
  import ultralytics
except Exception:
  print("Installing ultralytics, deep_sort_realtime, torchreid, motmetrics, and other deps...")

  # 🏆 FIX: Added torchreid to the installation command
  !pip install -q ultralytics deep_sort_realtime motmetrics torchreid

  !pip install torch==2.5.0 torchvision==0.20.0
  !pip install -q ftfy

# Note: versions may vary. If torchreid install fails, we'll fallback to torchvision-based embedder.

In [10]:
#imports

import cv2
import numpy as np
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
import math
import torch
import torch.nn as nn
import shutil
from pathlib import Path
import motmetrics as mm
from sklearn.metrics.pairwise import cosine_similarity


In [11]:
import shutil

if os.path.exists(CROPS_DIR):
    shutil.rmtree(CROPS_DIR)
os.makedirs(CROPS_DIR, exist_ok=True)
#This ensures no old tracks remain

In [12]:
def build_osnet_encoder(device='cuda' if torch.cuda.is_available() else 'cpu'):
  try:
      # Try using torchreid API
      import torchreid
      # torchreid has a convenient model_zoo; osnet_x1_0 is common
      model_name = 'osnet_x1_0'
      model = torchreid.models.build_model(
      name=model_name,
      num_classes=1000,
      loss='softmax',
      pretrained=True
      )
      # Remove classifier if present -> we want embeddings
      if hasattr(model, 'classifier'):
        model.classifier = nn.Identity()
      model.eval().to(device)

      def encode(images):
          # images: list of BGR numpy arrays (H,W,3)
          import torchvision.transforms as T
          transform = T.Compose([
            T.ToPILImage(),
            T.Resize((256, 128)),
            T.ToTensor(),
            T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
          ])
          tensors = [transform(cv2.cvtColor(img, cv2.COLOR_BGR2RGB)).unsqueeze(0) for img in images]
          batch = torch.cat(tensors, dim=0).to(device)
          with torch.no_grad():
            feats = model(batch)
          feats = feats.cpu().numpy()
          # L2 normalize
          norms = np.linalg.norm(feats, axis=1, keepdims=True)
          feats = feats / (norms + 1e-6)
          return feats

      print("OSNet encoder built using torchreid.")
      return encode
  except Exception as e:
    print("OSNet via torchreid not available or failed to load (falling back). Error:", e)
    return None


In [13]:
def build_resnet_encoder(device='cuda' if torch.cuda.is_available() else 'cpu'):
  import torchvision.models as models
  import torchvision.transforms as T
  resnet = models.resnet50(pretrained=True)
  # Remove final classifier
  modules = list(resnet.children())[:-1]
  feat_extractor = nn.Sequential(*modules)
  feat_extractor.eval().to(device)

  def encode(images):
    transform = T.Compose([
      T.ToPILImage(),
      T.Resize((256, 128)),
      T.ToTensor(),
      T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    tensors = [transform(cv2.cvtColor(img, cv2.COLOR_BGR2RGB)).unsqueeze(0) for img in images]
    batch = torch.cat(tensors, dim=0).to(device)
    with torch.no_grad():
      feats = feat_extractor(batch).squeeze(-1).squeeze(-1)
    feats = feats.cpu().numpy()
    norms = np.linalg.norm(feats, axis=1, keepdims=True)
    feats = feats / (norms + 1e-6)
    return feats


  print("ResNet50 encoder built (fallback).")
  return encode


In [14]:
# Build encoder: prefer OSNet
device = 'cuda' if torch.cuda.is_available() else 'cpu'
encoder = build_osnet_encoder(device)
if encoder is None:
  encoder = build_resnet_encoder(device)

'''orchreid is not installed on your environment
 So OSNet cannot load
So your code falls back to ResNet50 encoder

Why torchreid is missing?
Because torchreid does not support Python 3.12, and you're in Google Colab which uses:
Python 3.12
Torchreid only supports:
Python 3.7
Python 3.8'''


Successfully loaded imagenet pretrained weights from "/root/.cache/torch/checkpoints/osnet_x1_0_imagenet.pth"
OSNet encoder built using torchreid.


"orchreid is not installed on your environment\n So OSNet cannot load\nSo your code falls back to ResNet50 encoder\n\nWhy torchreid is missing?\nBecause torchreid does not support Python 3.12, and you're in Google Colab which uses:\nPython 3.12\nTorchreid only supports:\nPython 3.7\nPython 3.8"

In [15]:
# YOLO model
print("Loading YOLO model...")
yolo = YOLO(YOLO_WEIGHTS)

# DeepSORT initialization
tracker = DeepSort(
    max_age=30,
    n_init=3,
    max_iou_distance=0.7,
    max_cosine_distance=0.7,
    embedder='mobilenet',      # placeholder
    embedder_gpu=True
)


# ---- Custom Embedder (working for latest deep_sort_realtime) ----
class CustomEmbedder:
    def __init__(self, encode_fn, embedding_dim=512):
        self.encode_fn = encode_fn
        self.dim = embedding_dim

  # DeepSORT calls this
    def predict(self, crops):
        # crops = list of cropped images (BGR)
        if len(crops) == 0:
            return []
        return self.encode_fn(crops)

    def __call__(self, images, bboxes):
        crops = []
        for img, bbox in zip(images, bboxes):
            x, y, w, h = [int(v) for v in bbox]
            crop = img[y:y+h, x:x+w]

            if crop.size == 0:
                crop = cv2.resize(img, (128,256))
            crops.append(crop)

        return self.encode_fn(crops)


# Attach custom encoder
if encoder is not None:
    tracker.embedder = CustomEmbedder(encoder)
    print("Custom embedder is now active.")
else:
    print("Default embedder in use.")


Loading YOLO model...


/usr/local/lib/python3.12/dist-packages/deep_sort_realtime/embedder/embedder_pytorch.py:53: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.model.load_state_dict(torch.lo

Custom embedder is now active.


In [16]:
'''That means your YOLO model loaded successfully AND your custom OSNet/ResNet embedder is now fully attached to DeepSORT.

This means DeepSORT now uses:
YOLO → for detections
Your OSNet/ResNet encoder → for person/target re-identification
Kalman filter → for tracking over time'''

'That means your YOLO model loaded successfully AND your custom OSNet/ResNet embedder is now fully attached to DeepSORT.\n\nThis means DeepSORT now uses:\nYOLO → for detections\nYour OSNet/ResNet encoder → for person/target re-identification\nKalman filter → for tracking over time'

In [17]:
# ---------- Utilities ----------

def xyxy_to_xywh(box):
  x1, y1, x2, y2 = box
  w = x2 - x1
  h = y2 - y1
  return [x1, y1, w, h]

os.makedirs(CROPS_DIR, exist_ok=True)

# Save a crop and return path
def save_crop(img, frame_idx, track_id, crop_dir=CROPS_DIR):
  track_dir = Path(crop_dir) / f"id_{track_id}"
  track_dir.mkdir(parents=True, exist_ok=True)
  filename = track_dir / f"frame_{frame_idx:06d}.jpg"
  cv2.imwrite(str(filename), img)
  return str(filename)


In [18]:
!rm -rf /content/drive/MyDrive/tracks_crops/*


In [19]:
# ---------- Main processing loop: detect, track, save crops ----------
print("Starting video processing...")

cap = cv2.VideoCapture(VIDEO_PATH)
if not cap.isOpened():
    raise RuntimeError(f"Could not open video: {VIDEO_PATH}")

frame_idx = 0
mot_results = []  # store lines for MOT format
track_frame_index = {}  # dict: id -> list of frame indices
tracks_dict = {}  # frame -> {track_id: [l,t,r,b]}

while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame_idx += 1

    # YOLO inference
    res = yolo(frame, imgsz=640)[0]
    boxes = []
    confidences = []
    classes = []

    # Parse detections
    for det in res.boxes:
        cls = int(det.cls.cpu().numpy())
        if cls != 0:
            continue

        x1, y1, x2, y2 = map(int, det.xyxy[0].tolist())
        conf = float(det.conf.cpu().numpy())

        if conf < DETECTION_CONFIDENCE:
            continue

        boxes.append([x1, y1, x2, y2])
        confidences.append(conf)
        classes.append(cls)

    # Convert to DeepSORT format
    dets_for_tracker = []
    for b, c in zip(boxes, confidences):
        xywh = xyxy_to_xywh(b)
        dets_for_tracker.append((xywh, c, 'person'))

    # Update tracker
    tracks = tracker.update_tracks(dets_for_tracker, frame=frame)

    # ---- TRACK LOOP ----
    for tr in tracks:
        if not tr.is_confirmed():
            continue

        track_id = tr.track_id
        l, t, r, b = map(int, tr.to_ltrb())
        w = r - l
        h = b - t

        # -----------------------------
        # ✅ CROP SAVING (FULL BLOCK)
        # -----------------------------
        crop = frame[t:b, l:r]

        if crop.size != 0:
            track_dir = Path(CROPS_DIR) / f"id_{track_id}"
            track_dir.mkdir(parents=True, exist_ok=True)

            crop_path = track_dir / f"{frame_idx}.jpg"
            cv2.imwrite(str(crop_path), crop)
        else:
            print(f"Skipping empty crop: ID {track_id} frame {frame_idx}")

        # Track index list
        track_frame_index.setdefault(track_id, []).append(frame_idx)

        # Save to tracks_dict correctly
        tracks_dict.setdefault(frame_idx, {})[track_id] = [l, t, r, b]

        # MOT output
        mot_results.append(
            f"{frame_idx},{track_id},{l},{t},{w},{h},"
            f"{tr.det_confidence if hasattr(tr,'det_confidence') else 1.0},-1,-1"
        )

        # drawing
        cv2.rectangle(frame, (l, t), (r, b), (0, 255, 0), 2)
        cv2.putText(frame, f"ID:{track_id}", (l, t-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2)

print("Finished processing video.")
cap.release()

# Save MOT results
with open(MOT_RESULTS_PATH, "w") as f:
    f.write("\n".join(mot_results))

print(f"Tracker MOT-format results saved to: {MOT_RESULTS_PATH}")


Starting video processing...

0: 480x640 3 persons, 1 car, 1 truck, 1 bird, 72.3ms
Speed: 10.7ms preprocess, 72.3ms inference, 2.5ms postprocess per image at shape (1, 3, 480, 640)



/tmp/ipython-input-3994567460.py:27: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  cls = int(det.cls.cpu().numpy())
/tmp/ipython-input-3994567460.py:32: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  conf = float(det.conf.cpu().numpy())


0: 480x640 3 persons, 1 car, 1 truck, 1 bird, 10.4ms
Speed: 5.6ms preprocess, 10.4ms inference, 1.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 persons, 1 car, 1 truck, 1 bird, 10.8ms
Speed: 5.8ms preprocess, 10.8ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 persons, 1 car, 1 truck, 1 bird, 11.2ms
Speed: 5.5ms preprocess, 11.2ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 persons, 1 car, 1 truck, 1 bird, 7.1ms
Speed: 4.8ms preprocess, 7.1ms inference, 1.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 persons, 1 car, 1 truck, 1 bird, 7.1ms
Speed: 4.1ms preprocess, 7.1ms inference, 1.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 4 persons, 1 car, 1 truck, 1 bird, 7.1ms
Speed: 2.9ms preprocess, 7.1ms inference, 1.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 persons, 1 car, 1 truck, 1 bird, 8.0ms
Speed: 3.5ms preprocess, 8.0ms inference, 1.7ms 

In [21]:
# print("\nDEBUG: Number of embeddings:", len(track_embeddings))
# print("DEBUG: Embedding IDs:", list(track_embeddings.keys()))

# print("DEBUG: tracks_dict frames:", len(tracks_dict))
# print("DEBUG: Example 5 frames from tracks_dict:", list(tracks_dict.keys())[:5])

# # print("DEBUG: track_frame_index keys:", track_frame_index.keys())
# print("DEBUG: track_frame_index lens:", {tid: len(fr) for tid, fr in track_frame_index.items()})


In [22]:
print("\nCHECK: DeepSORT confirmed tracks:")
confirmed_ids = set()
for f, d in tracks_dict.items():
    confirmed_ids.update(d.keys())
print("Confirmed IDs:", confirmed_ids)



CHECK: DeepSORT confirmed tracks:
Confirmed IDs: {'2', '69', '1', '12', '21', '20', '6', '62', '61', '47', '51', '82', '16', '32', '9', '48', '70', '15', '3', '19', '18', '8', '54', '75', '13', '37', '50', '49', '17', '22'}


In [ ]:
# print("DEBUG: Crop folders found:", list(Path(CROPS_DIR).glob("*")))


In [ ]:
# #Converting track_frame_index keys are integers: 21, 48, 49, ...
# # tracks_dict keys (inside the loop) are strings: '125', '127', ...
# # Convert keys to int when building valid_track_ids:

# # valid_track_ids = set()
# for f, ids in tracks_dict.items():
#     valid_track_ids.update([int(k) for k in ids.keys()])


In [ ]:
# track_embeddings = {tid: emb for tid, emb in track_embeddings.items()
#                      if tid in valid_track_ids}


# tracks_dict = {int(f): v for f, v in tracks_dict.items()}



In [52]:
INPUT_PERSON_IMAGE ="/content/drive/MyDrive/blue_input.png"

In [53]:
# ---------- Person search: given input image, find best matching track ID ----------
print("Running person matching...")

# Read query image
query_img = cv2.imread(INPUT_PERSON_IMAGE)
if query_img is None:
    raise RuntimeError(f"Could not read query image: {INPUT_PERSON_IMAGE}")

# Build embedding for query image
query_feat = encoder([query_img])[0]

# Build per-track prototype embeddings by averaging saved crops' embeddings
track_embeddings = {}
for track_dir in sorted(Path(CROPS_DIR).glob('id_*')):
    img_paths = sorted(track_dir.glob('*.jpg'))
    imgs = [cv2.imread(str(p)) for p in img_paths if cv2.imread(str(p)) is not None]
    if len(imgs) == 0:
        continue

    feats = encoder(imgs)
    proto = feats.mean(axis=0)
    proto = proto / (np.linalg.norm(proto) + 1e-6)
    track_id = int(track_dir.name.split('_')[1])
    track_embeddings[track_id] = proto

print("DEBUG: Number of embeddings:", len(track_embeddings))
print("DEBUG: Embedding IDs:", list(track_embeddings.keys()))

# Convert frame keys in track_frame_index and tracks_dict to integers
track_frame_index = {int(k): v for k, v in track_frame_index.items()}
tracks_dict = {int(f): {int(k): v for k, v in frame_dict.items()} for f, frame_dict in tracks_dict.items()}

# Remove IDs that never appeared in tracks_dict
valid_track_ids = set()
for f, ids in tracks_dict.items():
    valid_track_ids.update(ids.keys())

track_embeddings = {tid: emb for tid, emb in track_embeddings.items() if tid in valid_track_ids}

print("Track embeddings after filtering valid IDs:", list(track_embeddings.keys()))

# Find best matching track
best_id = None
best_score = -1
for tid, emb in track_embeddings.items():
    # Only match against tracks that were actually tracked in the video
    if tid not in track_frame_index or len(track_frame_index[tid]) == 0:
        continue

    score = float(np.dot(query_feat, emb))  # cosine similarity
    if score > best_score:
        best_score = score
        best_id = tid

if best_id is None:
    raise RuntimeError("❌ No valid tracked persons found to match against.")

print(f"Best match -> Track ID: {best_id} with similarity {best_score:.4f}")


Running person matching...
DEBUG: Number of embeddings: 30
DEBUG: Embedding IDs: [1, 12, 13, 15, 16, 17, 18, 19, 2, 20, 21, 22, 3, 32, 37, 47, 48, 49, 50, 51, 54, 6, 61, 62, 69, 70, 75, 8, 82, 9]
Track embeddings after filtering valid IDs: [1, 12, 13, 15, 16, 17, 18, 19, 2, 20, 21, 22, 3, 32, 37, 47, 48, 49, 50, 51, 54, 6, 61, 62, 69, 70, 75, 8, 82, 9]
Best match -> Track ID: 18 with similarity 0.8434


In [54]:
OUTPUT_CLIP_PATH = "/content/drive/MyDrive/new_output.mp4"


In [56]:
print("Best ID:", best_id)
print("Frames where this ID appears:", len(track_frame_index.get(best_id, [])))
print("First 30 frames:", track_frame_index.get(best_id, [])[:30])


Best ID: 18
Frames where this ID appears: 136
First 30 frames: [117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146]


In [57]:
print("Best ID:", best_id)
print("Frames where best_id appears:", len(track_frame_index.get(best_id, [])))
print("First 30 frames of best_id:", track_frame_index.get(best_id, [])[:30])

# Show ALL track IDs and how many frames they exist in
print("\n=== Track lengths ===")
for tid, frames in track_frame_index.items():
    print("ID", tid, "->", len(frames), "frames")


Best ID: 18
Frames where best_id appears: 136
First 30 frames of best_id: [117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146]

=== Track lengths ===
ID 1 -> 134 frames
ID 2 -> 51 frames
ID 3 -> 47 frames
ID 6 -> 412 frames
ID 8 -> 407 frames
ID 9 -> 58 frames
ID 12 -> 50 frames
ID 13 -> 518 frames
ID 15 -> 380 frames
ID 16 -> 117 frames
ID 17 -> 125 frames
ID 18 -> 136 frames
ID 19 -> 185 frames
ID 20 -> 63 frames
ID 21 -> 648 frames
ID 22 -> 47 frames
ID 32 -> 214 frames
ID 37 -> 65 frames
ID 47 -> 130 frames
ID 48 -> 333 frames
ID 49 -> 297 frames
ID 50 -> 145 frames
ID 51 -> 290 frames
ID 54 -> 148 frames
ID 61 -> 38 frames
ID 62 -> 224 frames
ID 69 -> 168 frames
ID 70 -> 119 frames
ID 75 -> 85 frames
ID 82 -> 44 frames


In [58]:
cap = cv2.VideoCapture(VIDEO_PATH)
print("Video FPS:", cap.get(cv2.CAP_PROP_FPS))
cap.release()


Video FPS: 10.0


In [59]:
# print("Creating cropped-person-only video...")

# cap = cv2.VideoCapture(VIDEO_PATH)
# if not cap.isOpened():
#     raise RuntimeError(f"Could not open video: {VIDEO_PATH}")

# out_crop = None
# frame_idx = 0
# written_count = 0
# last_valid_box = None  # fallback in case of bad crops
# input_fps = 10  # set your real video FPS

# # Set desired fixed size for output video
# CROP_WIDTH, CROP_HEIGHT = 256, 512  # adjust as needed

# while True:
#     ret, frame = cap.read()
#     if not ret:
#         break
#     frame_idx += 1

#     if frame_idx not in tracks_dict:
#         continue
#     if best_id not in tracks_dict[frame_idx]:
#         continue

#     # Get bbox
#     l, t, r, b = tracks_dict[frame_idx][best_id]

#     # Clamp box to frame boundaries
#     l = max(0, l)
#     t = max(0, t)
#     r = min(frame.shape[1], r)
#     b = min(frame.shape[0], b)

#     # Ensure width and height are at least 1
#     if r <= l or b <= t:
#         if last_valid_box is not None:
#             l, t, r, b = last_valid_box
#         else:
#             print(f"Skipping frame {frame_idx} due to invalid box: {(l,t,r,b)}")
#             continue

#     crop = frame[t:b, l:r]
#     if crop.size == 0:
#         print(f"Bad crop at frame {frame_idx}: {(l,t,r,b)}")
#         continue

#     last_valid_box = (l, t, r, b)  # save last valid

#     # Resize crop to fixed size
#     crop_resized = cv2.resize(crop, (CROP_WIDTH, CROP_HEIGHT))

#     # Initialize video writer once we know size
#     if out_crop is None:
#         fourcc = cv2.VideoWriter_fourcc(*"mp4v")
#         out_crop = cv2.VideoWriter(
#             OUTPUT_CLIP_PATH,
#             fourcc,
#             input_fps,
#             (CROP_WIDTH, CROP_HEIGHT),
#             True
#         )

#     out_crop.write(crop_resized)
#     written_count += 1

# cap.release()
# if out_crop:
#     out_crop.release()

# print(f"Total frames written: {written_count}")
# print(f"Cropped video saved: {OUTPUT_CLIP_PATH}")


In [60]:
print("Creating cropped-person-only video...")

cap = cv2.VideoCapture(VIDEO_PATH)
if not cap.isOpened():
    raise RuntimeError(f"Could not open video: {VIDEO_PATH}")

out_crop = None
frame_idx = 0
written_count = 0
last_valid_box = None  # fallback in case of bad crops
input_fps = 10  # set your real video FPS

# Fixed output video size
CROP_WIDTH, CROP_HEIGHT = 256, 512  # adjust as needed
SIM_THRESHOLD = 0.7  # minimum similarity to query image

while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame_idx += 1

    # Skip frames if track not present
    if frame_idx not in tracks_dict:
        continue
    if best_id not in tracks_dict[frame_idx]:
        continue

    # Get bbox
    l, t, r, b = tracks_dict[frame_idx][best_id]

    # Clamp to frame boundaries
    l = max(0, l)
    t = max(0, t)
    r = min(frame.shape[1], r)
    b = min(frame.shape[0], b)

    if r <= l or b <= t:
        if last_valid_box is not None:
            l, t, r, b = last_valid_box
        else:
            print(f"Skipping frame {frame_idx} due to invalid box: {(l,t,r,b)}")
            continue

    crop = frame[t:b, l:r]
    if crop.size == 0:
        print(f"Bad crop at frame {frame_idx}: {(l,t,r,b)}")
        continue

    # Compute per-frame similarity to query image
    crop_feat = encoder([crop])[0]
    score = float(np.dot(query_feat, crop_feat))
    if score < SIM_THRESHOLD:
        # Skip frames that don't match well
        continue

    last_valid_box = (l, t, r, b)  # save last valid

    # Resize crop to fixed size
    crop_resized = cv2.resize(crop, (CROP_WIDTH, CROP_HEIGHT))

    # Initialize video writer once
    if out_crop is None:
        fourcc = cv2.VideoWriter_fourcc(*"mp4v")
        out_crop = cv2.VideoWriter(
            OUTPUT_CLIP_PATH,
            fourcc,
            input_fps,
            (CROP_WIDTH, CROP_HEIGHT),
            True
        )

    out_crop.write(crop_resized)
    written_count += 1

cap.release()
if out_crop:
    out_crop.release()

print(f"Total frames written: {written_count}")
print(f"Cropped video saved: {OUTPUT_CLIP_PATH}")


Creating cropped-person-only video...
Total frames written: 88
Cropped video saved: /content/drive/MyDrive/new_output.mp4


In [61]:
from google.colab import files

files.download("/content/drive/MyDrive/new_output.mp4")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [71]:
MOT_RESULTS_PATH = "/content/drive/MyDrive/mot_results.txt" # tracker output for MOT evaluation
GROUND_TRUTH_MOT = "/content/drive/MyDrive/gt.txt" # optional: ground truth in MOT format for evaluation


In [72]:
'''import os
from pathlib import Path
import cv2
import numpy as np
import pandas as pd
import motmetrics as mm

# -----------------------------
# SETTINGS
# -----------------------------
VIDEO_PATH ="/content/drive/MyDrive/color_input.mp4" # Input video

CROPS_DIR = "/content/drive/MyDrive/tracks_crops"
               # Folder where cropped frames per ID are saved
TRACKER_MOT_RESULTS = "/content/drive/MyDrive/mot_results.txt"  # Tracker MOT output
GT_FILE = "/content/drive/MyDrive/gt.txt"                   # Ground-truth file we will create
QUERY_TRACK_ID = 1                    # Track ID for query person
INTERVAL = 5                          # Annotate every INTERVAL frames
IOU_THRESHOLD = 0.5                   # Threshold to count a frame as correct

In [88]:
# @title
'''# @title
# 1️⃣ Generate ground truth annotation for the query person
# -----------------------------
print("Generating ground-truth annotations from cropped frames...")

# Gather cropped frames for the query person
query_crop_dir = Path(CROPS_DIR) / f"id_{QUERY_TRACK_ID}"
if not query_crop_dir.exists():
    raise RuntimeError(f"No crops found for track ID {QUERY_TRACK_ID}")

gt_entries = []

for i, img_path in enumerate(sorted(query_crop_dir.glob("*.jpg"))):
    if i % INTERVAL != 0:
        continue  # skip frames if interval > 1

    # Load image to get width and height
    img = cv2.imread(str(img_path))
    if img is None:
        print(f"Warning: Could not read image {img_path}, skipping")
        continue
    h, w = img.shape[:2]

    # Extract frame index from filename robustly
    stem = img_path.stem  # filename without extension
    try:
        frame_idx = int(stem)  # for filenames like 0.jpg, 1.jpg
    except ValueError:
        # for filenames like frame_0.jpg, frame_1.jpg
        if "frame_" in stem:
            frame_idx = int(stem.replace("frame_", ""))
        else:
            raise ValueError(f"Cannot extract frame index from filename: {img_path.name}")

    # For simplicity, assume bbox = full crop
    # Top-left at 0,0 in crop (full crop size)
    x, y = 0, 0
    gt_entries.append([frame_idx, QUERY_TRACK_ID, x, y, w, h, 1, -1, -1])

# Save GT in MOT format
gt_df = pd.DataFrame(gt_entries, columns=['frame','id','x','y','w','h','conf','xx','yy'])
gt_df.to_csv(GT_FILE, header=False, index=False)
print(f"Ground-truth annotations saved to: {GT_FILE}")


Generating ground-truth annotations from cropped frames...
Ground-truth annotations saved to: /content/drive/MyDrive/gt.txt


In [74]:
'''# @title
# 2️⃣ Load tracker MOT results
# -----------------------------
tracker_df = pd.read_csv(TRACKER_MOT_RESULTS, header=None)
tracker_df.columns = ['frame','id','x','y','w','h','conf','xx','yy']

In [82]:
# -----------------------------
# 3️⃣ MOT evaluation using manual IoU (compatible with NumPy 2.0+)
# -----------------------------

In [87]:
# @title
'''# @title
acc = mm.MOTAccumulator(auto_id=False)
iou_threshold = 0.5

def iou_matrix_manual(gt_boxes, ts_boxes):
    D = np.zeros((len(gt_boxes), len(ts_boxes)))
    for i, gt in enumerate(gt_boxes):
        x1g, y1g, wg, hg = gt
        x2g, y2g = x1g+wg, y1g+hg
        for j, ts in enumerate(ts_boxes):
            x1t, y1t, wt, ht = ts
            x2t, y2t = x1t+wt, y1t+ht
            xi1 = max(x1g, x1t)
            yi1 = max(y1g, y1t)
            xi2 = min(x2g, x2t)
            yi2 = min(y2g, y2t)
            iw = max(0, xi2 - xi1)
            ih = max(0, yi2 - yi1)
            inter = iw * ih
            union = wg*hg + wt*ht - inter
            D[i,j] = 1 - (inter/union) if union>0 else 1.0
    return D

frames = sorted(set(gt_df['frame']).union(tracker_df['frame']))

for f in frames:
    gt_frame = gt_df[gt_df['frame']==f]
    ts_frame = tracker_df[tracker_df['frame']==f]

    gt_ids = gt_frame['id'].tolist()
    ts_ids = ts_frame['id'].tolist()

    gt_boxes = gt_frame[['x','y','w','h']].values.tolist()
    ts_boxes = ts_frame[['x','y','w','h']].values.tolist()

    if len(gt_boxes)==0 or len(ts_boxes)==0:
        D = np.zeros((len(gt_boxes), len(ts_boxes)))
    else:
        D = iou_matrix_manual(gt_boxes, ts_boxes)
        D[D > 1 - iou_threshold] = np.inf

    # Pass frameid explicitly because auto_id=False
    acc.update(gt_ids, ts_ids, D, frameid=f)

mh = mm.metrics.create()
summary = mh.compute(
    acc,
    metrics=['num_frames','mota','motp','precision','recall'],
    name='acc'
)
print("\nMOT evaluation summary:")
print(summary)



MOT evaluation summary:
     num_frames        mota  motp  precision  recall
acc         793 -210.296296   NaN        0.0     0.0


In [1]:
import cv2
import pandas as pd
from pathlib import Path

VIDEO_PATH ="/content/drive/MyDrive/color_input.mp4"     # Path to your video
OUTPUT_CSV = "ground_truth.csv"    # Ground truth output
LABEL_ID = 1                        # ID for your query person
FRAME_STEP = 5                       # Annotate every 5th frame (adjust as needed)

gt_entries = []

In [2]:
# Callback to draw rectangle
# -----------------------------
drawing = False
ix, iy = -1, -1
current_frame = None
frame_idx = 0

def draw_rectangle(event, x, y, flags, param):
    global ix, iy, drawing, current_frame, temp_frame
    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        ix, iy = x, y
        temp_frame[:] = current_frame.copy()
    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing:
            temp_frame[:] = current_frame.copy()
            cv2.rectangle(temp_frame, (ix, iy), (x, y), (0, 255, 0), 2)
    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        x1, y1 = min(ix, x), min(iy, y)
        x2, y2 = max(ix, x), max(iy, y)
        w, h = x2 - x1, y2 - y1
        gt_entries.append([frame_idx, LABEL_ID, x1, y1, w, h, 1, -1, -1])
        cv2.rectangle(temp_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

In [ ]:
# Load video
# -----------------------------
cap = cv2.VideoCapture(VIDEO_PATH)
if not cap.isOpened():
    raise RuntimeError(f"Cannot open video {VIDEO_PATH}")

cv2.namedWindow("Annotate")
cv2.setMouseCallback("Annotate", draw_rectangle)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame_idx += 1

    # Skip frames according to FRAME_STEP
    if frame_idx % FRAME_STEP != 1:
        continue

    current_frame = frame.copy()
    temp_frame = frame.copy()

    while True:
        cv2.imshow("Annotate", temp_frame)
        key = cv2.waitKey(1) & 0xFF

        if key == ord("n"):  # Next frame
            break
        elif key == ord("r"):  # Reset annotations for current frame
            temp_frame[:] = current_frame.copy()
        elif key == ord("q"):  # Quit and save annotations
            cap.release()
            cv2.destroyAllWindows()
            gt_df = pd.DataFrame(
                gt_entries,
                columns=['frame','id','x','y','w','h','conf','xx','yy']
            )
            gt_df.to_csv(OUTPUT_CSV, index=False, header=False)
            print(f"Ground-truth saved to {OUTPUT_CSV}")
            exit(0)

cap.release()
cv2.destroyAllWindows()

# Save final annotations
gt_df = pd.DataFrame(
    gt_entries,
    columns=['frame','id','x','y','w','h','conf','xx','yy']
)
gt_df.to_csv(OUTPUT_CSV, index=False, header=False)
print(f"Ground-truth saved to {OUTPUT_CSV}")

In [98]:
'''# -----------------------------
# 4️⃣ Optional: Frame-level accuracy using IoU
# -----------------------------
def iou(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    interW = max(0, xB - xA)
    interH = max(0, yB - yA)
    interArea = interW * interH
    boxAArea = (boxA[2]-boxA[0]) * (boxA[3]-boxA[1])
    boxBArea = (boxB[2]-boxB[0]) * (boxB[3]-boxB[1])
    if boxAArea + boxBArea - interArea == 0:
        return 0.0
    return interArea / (boxAArea + boxBArea - interArea)

correct = 0
total = len(gt_df)
for idx, row in gt_df.iterrows():
    f = row['frame']
    gt_box = [row['x'], row['y'], row['x']+row['w'], row['y']+row['h']]
    ts_row = tracker_df[(tracker_df['frame']==f) & (tracker_df['id']==row['id'])]
    if not ts_row.empty:
        pred_box = ts_row.iloc[0]
        pred_box = [pred_box['x'], pred_box['y'], pred_box['x']+pred_box['w'], pred_box['y']+pred_box['h']]
        if iou(gt_box, pred_box) >= IOU_THRESHOLD:
            correct += 1

accuracy = correct / total if total>0 else 0.0
print(f"\nSingle-person frame-level IoU accuracy (threshold={IOU_THRESHOLD}): {accuracy*100:.2f}%")


Single-person frame-level IoU accuracy (threshold=0.5): 100.00%
